In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

import ktrain
from ktrain import text
import time
import pandas as pd

/home/roguehydra/.local/lib/python3.6/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# variables
max_len = 100

trn, val, preproc = text.texts_from_folder('../../Dataset/Data/Ktrain/TOFU-BERT', 
                                            maxlen=max_len, 
                                            preprocess_mode='distilbert',
                                            train_test_names=['train', 'test'],
                                            classes=['pos', 'neg'])

detected encoding: utf-8
preprocessing train...
language: en
train sequence lengths:
	mean : 53
	95percentile : 155
	99percentile : 280


preprocessing test...
language: en
test sequence lengths:
	mean : 35
	95percentile : 92
	99percentile : 152


In [3]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

Is Multi-Label? False
maxlen is 100
done.


In [10]:
# 2e-5 is one of the LRs recommended by Google
b = time.time()
epochs = 1
l_r = 2e-5
learner.fit_onecycle(l_r, epochs)
e = time.time()
print(e-b)



begin training using onecycle policy with max lr of 2e-05...
Train for 425 steps, validate for 17 steps
425/425 [==============================] - 837s 2s/step - loss: 0.4639 - accuracy: 0.7693 - val_loss: 0.3726 - val_accuracy: 0.8521
840.1992373466492


In [24]:
df = pd.read_excel('../../Dataset/Data/TOFU/Test - clean.xlsx')
test_x = list(df.Text)
test_y = df.Sentiment
predictor = ktrain.get_predictor(learner.model, preproc)
b = time.time()
pred = predictor.predict(test_x)
e = time.time()
print(round(e-b,2))

41.48


In [31]:
score = 0
for i in range(len(test_y)):
    if pred[i] == 'pos' and test_y[i] == 1:
        score += 1
    elif pred[i] == 'neg' and test_y[i] == 0:
        score += 1
print('Accuracy: {}'.format(round(score/len(test_y)*100,2)))

Accuracy: 85.02


In [32]:
predictor.save('tofu-distilbertpredictor_acc:85%')

In [37]:
predictor = ktrain.load_predictor('tofu-distilbertpredictor_acc:85%')